In [ ]:
import os
import sys
import pandas as pd
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
sns.set()

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
from relnet.evaluation.storage import EvaluationStorage
from relnet.evaluation.file_paths import FilePaths
from relnet.agent.mcts.mcts_agent import MonteCarloTreeSearchAgent


algorithm_class = 'planning'
experiment_id = 'prelim_kh_25'


storage = EvaluationStorage()
fp = FilePaths('/experiment_data', 'aggregate')

agent_hyperparam_dfs = {}
param_spaces, df = storage.get_hyperparameter_optimisation_data(algorithm_class, experiment_id, {}, train_individually=False)

latest_experiment = storage.get_experiment_details(algorithm_class, experiment_id)
agent_names = latest_experiment["agents"]
network_generators = latest_experiment["network_generators"]
objective_functions = latest_experiment["objective_functions"]

for agent_name in set(df['agent_name']):
    expanded_data = []
    for objective_function in objective_functions:
        subset = df[(df['agent_name'] == agent_name) & (df['objective_function'] == objective_function)]
        subset.drop(columns=['agent_name'])

        for idx, row in subset.iterrows():
            row_copy = dict(row)
            hyperparams_id = row['hyperparams_id']
            hyperparams = param_spaces[objective_function][agent_name][hyperparams_id]
            row_copy.update(hyperparams)
            expanded_data.append(row_copy)
    
    hyp_df = pd.DataFrame(expanded_data).drop(columns=['hyperparams_id'])
    hyp_df["adjust_C_p"] = hyp_df["adjust_C_p"].astype(int)
    agent_hyperparam_dfs[agent_name] = hyp_df



In [ ]:
for agent_name in agent_names:
    if agent_name not in agent_hyperparam_dfs:
        continue
    hyperparams_df = agent_hyperparam_dfs[agent_name]
    separate_per_budgets = False

    for network_generator in network_generators:
        for objective_function in objective_functions:  
            filtered_df = hyperparams_df[(hyperparams_df['network_generator'] == network_generator) &
                                        (hyperparams_df['objective_function'] == objective_function)]

            non_hyp_cols = {"agent_name", "avg_reward", "ar_fun", "ar_modifier", "network_generator", "objective_function", "graph_id", "rollout_depth", "adjust_C_p"}
            hyperparam_cols = list(set(hyperparams_df.columns) - non_hyp_cols)
            for hyperparam_name in hyperparam_cols:
                plt.figure()
                title = f"{agent_name}-{network_generator}-{objective_function}-{hyperparam_name}-all"
                filename = f"{experiment_id}-hyperparams-{title}.pdf"
                plt.title(title)
                sns.lineplot(data=filtered_df, x=hyperparam_name, y="avg_reward")
                plt.savefig(fp.figures_dir / filename, bbox_inches='tight')
                plt.close()

            if separate_per_budgets:
                budgets = sorted(list(set(hyperparams_df['expansion_budget_modifier'])))
                for hyperparam_name in hyperparam_cols:
                    for budget in budgets:
                        budget_filtered_df = filtered_df[filtered_df["expansion_budget_modifier"] == budget]
                        print(f"doing col {hyperparam_name}")
                        plt.figure()
                        title = f"{agent_name}-{network_generator}-{objective_function}-{hyperparam_name}-{budget}"
                        filename = f"{experiment_id}-hyperparams-{title}.pdf"
                        plt.title(title)
                        sns.lineplot(data=budget_filtered_df, x=hyperparam_name, y="avg_reward")
                        plt.savefig(fp.figures_dir / filename, bbox_inches='tight')
                        plt.close()

            